In [1]:
import ROOT
import numpy as np

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x634c6b0


In [2]:
%jsroot on

In [3]:
# FIXME: don't touch! the fortran code has a global state :(
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
# plot_path = "plots/angles/mg5_1M"
plot_path = "/dev/null"
oo_plot_name = "OO_wz.pdf"

In [5]:
ROOT.gSystem.Load("OO/diehl/libObservable.so")

0

In [6]:
ROOT.gInterpreter.Declare("#include <WWTools.h>")
ROOT.gInterpreter.Declare("#include \"OO/diehl/OOTools.h\"")
ROOT.gInterpreter.Declare("#include <podio/GenericParameters.h>")

True

In [7]:
# use same masses as in whizard!
mW = 80.419
mZ = 91.1882

# ROOT.OOTools.init(250, mW, mZ)
ROOT.OOTools.init(190, mW, mZ)

In [8]:
ROOT.ildStyle.SetOptStat(1)
# ROOT.ildStyle.SetOptStat(0)

In [9]:
%%cpp
using namespace ROOT::VecOps;

In [10]:
# df = ROOT.RDataFrame("events", "data/gen/whizard_ww_semi_250/ww_ac_e-1.edm4hep.root")
# df = ROOT.RDataFrame("events", "data/gen/whizard_ww_semi_190/ww_ac_e-1.edm4hep.root")
df = ROOT.RDataFrame("events", "data/gen/whizard_ww_semi_190_oldM/ww_ac_e-1.edm4hep.root")
n_events = 1000000
df = df.Range(n_events)

In [11]:

df = df.Define("Parameters", "podio::GenericParameters par; par.loadFrom(GPDoubleKeys, GPDoubleValues); par.loadFrom(GPFloatKeys, GPFloatValues); par.loadFrom(GPIntKeys, GPIntValues); par.loadFrom(GPStringKeys, GPStringValues); return par;")
df = df.Define("weight", "(double) Parameters.get<float>(\"_weight\").value()")
df = df.Define("weight_alt1", "Parameters.get<double>(\"weight_alt1\").value()")
df = df.Define("weight_alt2", "Parameters.get<double>(\"weight_alt2\").value()")
df = df.Define("weight_alt3", "Parameters.get<double>(\"weight_alt3\").value()")

In [12]:
df = df.Vary("weight", "RVec<double>{weight * weight_alt1, weight * weight_alt2, weight * weight_alt3}", ["g1z_up", "ka_up", "la_up"], "aTGC")

In [13]:
# these files are very simple, they have the Ws explicitly and there is only one stable e/p
df = df.Define("e_MC_idx", "ArgMax(MCParticles.PDG == 11 && MCParticles.generatorStatus == 1)")
df = df.Define("enu_MC_idx", "ArgMax(MCParticles.PDG == -12 && MCParticles.generatorStatus == 1)")
df = df.Define("p_MC_idx", "ArgMax(MCParticles.PDG == -1 && MCParticles.generatorStatus == 1)")
df = df.Define("pnu_MC_idx", "ArgMax(MCParticles.PDG == 2 && MCParticles.generatorStatus == 1)")
df = df.Define("Wp_MC_idx", "ArgMax(MCParticles.PDG == 24)")
df = df.Define("Wm_MC_idx", "ArgMax(MCParticles.PDG == -24)")

df = df.Define("e_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[e_MC_idx], MCParticles.momentum.y[e_MC_idx], MCParticles.momentum.z[e_MC_idx], MCParticles.mass[e_MC_idx])")
df = df.Define("enu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[enu_MC_idx], MCParticles.momentum.y[enu_MC_idx], MCParticles.momentum.z[enu_MC_idx], MCParticles.mass[enu_MC_idx])")
df = df.Define("p_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[p_MC_idx], MCParticles.momentum.y[p_MC_idx], MCParticles.momentum.z[p_MC_idx], MCParticles.mass[p_MC_idx])")
df = df.Define("pnu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[pnu_MC_idx], MCParticles.momentum.y[pnu_MC_idx], MCParticles.momentum.z[pnu_MC_idx], MCParticles.mass[pnu_MC_idx])")
df = df.Define("Wp_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[Wp_MC_idx], MCParticles.momentum.y[Wp_MC_idx], MCParticles.momentum.z[Wp_MC_idx], MCParticles.mass[Wp_MC_idx])")
df = df.Define("Wm_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[Wm_MC_idx], MCParticles.momentum.y[Wm_MC_idx], MCParticles.momentum.z[Wm_MC_idx], MCParticles.mass[Wm_MC_idx])")

In [14]:

df = df.Define("l_MC_lvec_e", "e_MC_lvec.energy()")
df = df.Define("l_MC_lvec_theta", "e_MC_lvec.theta()")
df = df.Define("l_MC_lvec_cosTheta", "cos(l_MC_lvec_theta)")

df = df.Define("l_MC_lvec_pt", "e_MC_lvec.pt()")
df = df.Define("l_MC_lvec_pz", "e_MC_lvec.pz()")

df = df.Define("e_MC_lvec_e", "e_MC_lvec.energy()")
df = df.Define("enu_MC_lvec_e", "enu_MC_lvec.energy()")
df = df.Define("p_MC_lvec_e", "p_MC_lvec.energy()")
df = df.Define("pnu_MC_lvec_e", "pnu_MC_lvec.energy()")

df = df.Define("e_MC_lvec_px", "e_MC_lvec.px()")
df = df.Define("e_MC_lvec_py", "e_MC_lvec.py()")
df = df.Define("e_MC_lvec_pz", "e_MC_lvec.pz()")

df = df.Define("p_MC_lvec_px", "p_MC_lvec.px()")
df = df.Define("p_MC_lvec_py", "p_MC_lvec.py()")
df = df.Define("p_MC_lvec_pz", "p_MC_lvec.pz()")

df = df.Define("Wm_MC_lvec_px", "Wm_MC_lvec.px()")
df = df.Define("Wm_MC_lvec_py", "Wm_MC_lvec.py()")
df = df.Define("Wm_MC_lvec_pz", "Wm_MC_lvec.pz()")

df = df.Define("Wp_MC_lvec_px", "Wp_MC_lvec.px()")
df = df.Define("Wp_MC_lvec_py", "Wp_MC_lvec.py()")
df = df.Define("Wp_MC_lvec_pz", "Wp_MC_lvec.pz()")

### start of angle calculations
# the simplest is the production angle theta_W (where W stands not for Weinberg in this case but for W-boson)
# following M. Bilenky et al. in Nucl.Phys.B 409 (1993) 22-68 (doi: https://doi.org/10.1016/0550-3213(93)90445-U)
# this is the angle between the W^{-} and the beam e^{-} which in our case is just the Z-axis
# (TODO: check what happens when ddsim adds a crossing angle :/)

df = df.Define("W_minus_lvec", "Wm_MC_lvec")
df = df.Define("W_plus_lvec", "Wp_MC_lvec")
df = df.Define("e_minus_vec", "ROOT::Math::XYZVector(0, 0, 1)")
df = df.Define("e_plus_vec", "ROOT::Math::XYZVector(0, 0, -1)")

df = df.Define("CosTheta_W_minus", "ROOT::Math::VectorUtil::CosTheta(W_minus_lvec, e_minus_vec)")
# FIXME: e_minus_vec?
df = df.Define("CosTheta_W_plus", "ROOT::Math::VectorUtil::CosTheta(W_plus_lvec, e_minus_vec)")


In [15]:
df = df.Define("enu_m", "(e_MC_lvec + enu_MC_lvec).M()")
df = df.Define("pnu_m", "(p_MC_lvec + pnu_MC_lvec).M()")

In [16]:
# df = df.Define("e_transformed_lvec", "WWTools::starVectorOO(W_minus_lvec, e_MC_lvec)")
# df = df.Define("p_transformed_lvec", "WWTools::starVectorOO(W_plus_lvec, p_MC_lvec)")
# df = df.Define("e_transformed_lvec", "WWTools::starVector(W_minus_lvec, e_MC_lvec)")
# df = df.Define("e_transformed_lvec", "WWTools::starVector(W_minus_lvec, e_MC_lvec, {0, 0, 1})")
# df = df.Define("p_transformed_lvec", "WWTools::starVector(W_plus_lvec, p_MC_lvec)")
df = df.Define("e_transformed_lvec", "WWTools::starVectorHagiwara(W_minus_lvec, e_MC_lvec, {0, 0, 1}, W_minus_lvec)")
df = df.Define("p_transformed_lvec", "WWTools::starVectorHagiwara(W_plus_lvec, p_MC_lvec, {0, 0, 1}, W_minus_lvec)")
df = df.Define("pnu_transformed_lvec", "WWTools::starVectorHagiwara(W_plus_lvec, pnu_MC_lvec, {0, 0, 1}, W_minus_lvec)")

In [17]:

df = df.Define("e_theta_prime", "e_transformed_lvec.Theta()")
df = df.Define("e_phi_prime", "e_transformed_lvec.Phi()")
# df = df.Define("e_phi_prime", "e_transformed_lvec.Phi() + ROOT::Math::Pi()")
df = df.Define("p_theta_prime", "p_transformed_lvec.Theta()")
df = df.Define("p_phi_prime", "p_transformed_lvec.Phi()")
# df = df.Define("p_phi_prime", "p_transformed_lvec.Phi() + ROOT::Math::Pi()")
# df = df.Define("l_theta_prime", "auto theta = l_transformed_lvec.Theta(); return l_MC_charge == 1. ? ROOT::Math::Pi() - theta : theta;")
# df = df.Define("l_phi_prime", "auto phi = l_transformed_lvec.Phi(); return l_MC_charge == 1. ? phi - ROOT::Math::Pi() : phi;")

df = df.Define("pnu_theta_prime", "pnu_transformed_lvec.Theta()")
df = df.Define("pnu_phi_prime", "pnu_transformed_lvec.Phi()")

# OPAL shift W plus by 180 deg, but this gives it the wrong shape?!
# df = df.Define("l_phi_prime", "auto phi = l_transformed_lvec.Phi(); auto phi_s = l_MC_charge == 1. ? phi + ROOT::Math::Pi() : phi; return phi_s > ROOT::Math::Pi() ? phi_s - 2 * ROOT::Math::Pi() : phi_s;")
# df = df.Define("p_phi_prime", "auto phi = p_transformed_lvec.Phi() + ROOT::Math::Pi(); return phi > ROOT::Math::Pi() ? phi - 2 * ROOT::Math::Pi() : phi;")

df = df.Define("e_cosTheta_prime", "cos(e_theta_prime)")
df = df.Define("p_cosTheta_prime", "cos(p_theta_prime)")
df = df.Define("pnu_cosTheta_prime", "cos(pnu_theta_prime)")
# df = df.Define("p_cosTheta_prime", "-cos(p_theta_prime)")

In [18]:
# df = df.Define("aw_jet1", "OOTools::aw(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("awph_jet1", "OOTools::awph(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("abph_jet1", "OOTools::abph(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")

# df = df.Define("dg1z_jet1", "OOTools::dg1z(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("dkg_jet1", "OOTools::dkg(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("lz_jet1", "OOTools::lz(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")

# df = df.Define("fg1_jet1", "OOTools::fg1(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("fg2_jet1", "OOTools::fg2(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("fg3_jet1", "OOTools::fg3(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("fg4_jet1", "OOTools::fg4(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("fg5_jet1", "OOTools::fg5(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("fg6_jet1", "OOTools::fg6(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("fg7_jet1", "OOTools::fg7(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")

# df = df.Define("fz1_jet1", "OOTools::fz1(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("fz2_jet1", "OOTools::fz2(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("fz3_jet1", "OOTools::fz3(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("fz4_jet1", "OOTools::fz4(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("fz5_jet1", "OOTools::fz5(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("fz6_jet1", "OOTools::fz6(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
# df = df.Define("fz7_jet1", "OOTools::fz7(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")

# df = df.Define("aw_jet2", "OOTools::aw(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("awph_jet2", "OOTools::awph(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("abph_jet2", "OOTools::abph(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")

# df = df.Define("dg1z_jet2", "OOTools::dg1z(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("dkg_jet2", "OOTools::dkg(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("lz_jet2", "OOTools::lz(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")

# df = df.Define("fg1_jet2", "OOTools::fg1(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("fg2_jet2", "OOTools::fg2(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("fg3_jet2", "OOTools::fg3(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("fg4_jet2", "OOTools::fg4(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("fg5_jet2", "OOTools::fg5(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("fg6_jet2", "OOTools::fg6(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("fg7_jet2", "OOTools::fg7(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")

# df = df.Define("fz1_jet2", "OOTools::fz1(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("fz2_jet2", "OOTools::fz2(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("fz3_jet2", "OOTools::fz3(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("fz4_jet2", "OOTools::fz4(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("fz5_jet2", "OOTools::fz5(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("fz6_jet2", "OOTools::fz6(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
# df = df.Define("fz7_jet2", "OOTools::fz7(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")

for var in ["aw", "awph", "abph", "dg1z", "dkg", "lz"] + [f"fg{i}" for i in range(1,8)] + [f"fz{i}" for i in range(1,8)]:
    df = df.Define(f"{var}_jet1", f"OOTools::{var}(CosTheta_W_minus, e_cosTheta_prime, p_cosTheta_prime, e_phi_prime, p_phi_prime)")
    df = df.Define(f"{var}_jet2", f"OOTools::{var}(CosTheta_W_minus, e_cosTheta_prime, pnu_cosTheta_prime, e_phi_prime, pnu_phi_prime)")
    # FIXME this is of course not the same as taking both values and weighting them by 0.5... :(
    # df = df.Define(var, f"({var}_jet1 + {var}_jet2) * 0.5")
    # Diehl paper says its also ok to take the leading one if the OO (S1_jet1 + S1_jet2) / (S0_jet1 + S0_jet2)
    # which it effectively is due to the -co2, ph2 + Pi term
    # so I can just choose one of the two however I want as both "should" return the same
    # at least on generator level with no magic and only CC03...
    df = df.Define(var, f"{var}_jet1")

In [19]:
# FIXME: for debugging
# df = df.Define("weight", "1")
# df = df.Filter("l_MC_lvec_pt > 2", "pt cut")
# df = df.Filter("abs(l_MC_lvec_cosTheta) < 0.975", "cosTheta cut") # ~6.5 deg
# df = df.Filter(f"abs(enu_m - {mw}) < 2. && abs(pnu_m - {mw}) < 2.")

In [20]:
oo_bins = 200
# oo_bins = 25
# oo_bins = 250
oo_start = -3.
oo_end = 3.

h_aw = df.Histo1D(("", ";O_{#alphaW}", oo_bins, oo_start, oo_end), "aw", "weight")
h_awph = df.Histo1D(("", ";O_{#alphaW#phi}", oo_bins, oo_start, oo_end), "awph", "weight")
h_abph = df.Histo1D(("", ";O_{#alphaB#phi}", oo_bins, oo_start, oo_end), "abph", "weight")

h_dg1z = df.Histo1D(("", ";O_{#delta g_{1}^{Z}}", oo_bins, oo_start, oo_end), "dg1z", "weight")
h_dkg = df.Histo1D(("", ";O_{#delta#kappa_{#gamma}}", oo_bins, oo_start, oo_end), "dkg", "weight")
h_lz = df.Histo1D(("", ";O_{#lambda_{Z}}", oo_bins, oo_start, oo_end), "lz", "weight")

h_fg1 = df.Histo1D(("", ";O_{f_{1}^{#gamma}}", oo_bins, oo_start, oo_end), "fg1", "weight")
h_fg2 = df.Histo1D(("", ";O_{f_{2}^{#gamma}}", oo_bins, oo_start, oo_end), "fg2", "weight")
h_fg3 = df.Histo1D(("", ";O_{f_{3}^{#gamma}}", oo_bins, oo_start, oo_end), "fg3", "weight")
h_fg4 = df.Histo1D(("", ";O_{f_{4}^{#gamma}}", oo_bins, oo_start, oo_end), "fg4", "weight")
h_fg5 = df.Histo1D(("", ";O_{f_{5}^{#gamma}}", oo_bins, oo_start, oo_end), "fg5", "weight")
h_fg6 = df.Histo1D(("", ";O_{f_{6}^{#gamma}}", oo_bins, oo_start, oo_end), "fg6", "weight")
h_fg7 = df.Histo1D(("", ";O_{f_{7}^{#gamma}}", oo_bins, oo_start, oo_end), "fg7", "weight")

h_fz1 = df.Histo1D(("", ";O_{f_{1}^{Z}}", oo_bins, oo_start, oo_end), "fz1", "weight")
h_fz2 = df.Histo1D(("", ";O_{f_{2}^{Z}}", oo_bins, oo_start, oo_end), "fz2", "weight")
h_fz3 = df.Histo1D(("", ";O_{f_{3}^{Z}}", oo_bins, oo_start, oo_end), "fz3", "weight")
h_fz4 = df.Histo1D(("", ";O_{f_{4}^{Z}}", oo_bins, oo_start, oo_end), "fz4", "weight")
h_fz5 = df.Histo1D(("", ";O_{f_{5}^{Z}}", oo_bins, oo_start, oo_end), "fz5", "weight")
h_fz6 = df.Histo1D(("", ";O_{f_{6}^{Z}}", oo_bins, oo_start, oo_end), "fz6", "weight")
h_fz7 = df.Histo1D(("", ";O_{f_{7}^{Z}}", oo_bins, oo_start, oo_end), "fz7", "weight")

In [21]:
v_aw = ROOT.RDF.Experimental.VariationsFor(h_aw)
v_awph = ROOT.RDF.Experimental.VariationsFor(h_awph)
v_abph = ROOT.RDF.Experimental.VariationsFor(h_abph)

v_dg1z = ROOT.RDF.Experimental.VariationsFor(h_dg1z)
v_dkg = ROOT.RDF.Experimental.VariationsFor(h_dkg)
v_lz = ROOT.RDF.Experimental.VariationsFor(h_lz)

v_fg1 = ROOT.RDF.Experimental.VariationsFor(h_fg1)
v_fg2 = ROOT.RDF.Experimental.VariationsFor(h_fg2)
v_fg3 = ROOT.RDF.Experimental.VariationsFor(h_fg3)
v_fg4 = ROOT.RDF.Experimental.VariationsFor(h_fg4)
v_fg5 = ROOT.RDF.Experimental.VariationsFor(h_fg5)
v_fg6 = ROOT.RDF.Experimental.VariationsFor(h_fg6)
v_fg7 = ROOT.RDF.Experimental.VariationsFor(h_fg7)

v_fz1 = ROOT.RDF.Experimental.VariationsFor(h_fz1)
v_fz2 = ROOT.RDF.Experimental.VariationsFor(h_fz2)
v_fz3 = ROOT.RDF.Experimental.VariationsFor(h_fz3)
v_fz4 = ROOT.RDF.Experimental.VariationsFor(h_fz4)
v_fz5 = ROOT.RDF.Experimental.VariationsFor(h_fz5)
v_fz6 = ROOT.RDF.Experimental.VariationsFor(h_fz6)
v_fz7 = ROOT.RDF.Experimental.VariationsFor(h_fz7)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xc19a520


In [22]:
h_enu_m = df.Histo1D("enu_m")
h_pnu_m = df.Histo1D("pnu_m")

In [23]:
h_l_e = df.Histo1D(("", ";E_{e} [GeV]", 300, 0., 150.), "l_MC_lvec_e", "weight")
h_nu_e = df.Histo1D(("", ";E_{#nu} [GeV]", 300, 0., 150.), "enu_MC_lvec_e", "weight")
h_2d_e_e = df.Histo2D(("", ";E_{e-} [GeV];E_{#bar{#nu}} [GeV]", 300, 0., 150., 300, 0., 150.), "e_MC_lvec_e", "enu_MC_lvec_e", "weight")
h_2d_p_e = df.Histo2D(("", ";E_{e+} [GeV];E_{#nu} [GeV]", 300, 0., 150., 300, 0., 150.), "p_MC_lvec_e", "pnu_MC_lvec_e", "weight")
h_2d_lnu_pzpt = df.Histo2D(("", ";P_{z}(e) [GeV];P_{t}(e) [GeV]", 150, -150., 150., 150, 0., 150.), "l_MC_lvec_pz", "l_MC_lvec_pt", "weight")
h_2d_lnu_cosThetapt = df.Histo2D(("", ";cos #theta(e);P_{t}(e) [GeV]", 150, -1., 1., 150, 0., 150.), "l_MC_lvec_cosTheta", "l_MC_lvec_pt", "weight")

h_2d_ep_px = df.Histo2D(("", ";P_{x}(e-) [GeV];P_{x}(e+) [GeV]", 600, -150., 150., 600, -150., 150.), "e_MC_lvec_px", "p_MC_lvec_px", "weight")
h_2d_ep_py = df.Histo2D(("", ";P_{y}(e-) [GeV];P_{y}(e+) [GeV]", 600, -150., 150., 600, -150., 150.), "e_MC_lvec_py", "p_MC_lvec_py", "weight")
h_2d_ep_pz = df.Histo2D(("", ";P_{z}(e-) [GeV];P_{z}(e+) [GeV]", 600, -150., 150., 600, -150., 150.), "e_MC_lvec_pz", "p_MC_lvec_pz", "weight")

h_2d_WW_px = df.Histo2D(("", ";P_{x}(W-) [GeV];P_{x}(W+) [GeV]", 600, -150., 150., 600, -150., 150.), "Wm_MC_lvec_px", "Wp_MC_lvec_px", "weight")
h_2d_WW_py = df.Histo2D(("", ";P_{y}(W-) [GeV];P_{y}(W+) [GeV]", 600, -150., 150., 600, -150., 150.), "Wm_MC_lvec_py", "Wp_MC_lvec_py", "weight")
h_2d_WW_pz = df.Histo2D(("", ";P_{z}(W-) [GeV];P_{z}(W+) [GeV]", 600, -150., 150., 600, -150., 150.), "Wm_MC_lvec_pz", "Wp_MC_lvec_pz", "weight")

h_e_px = df.Histo1D(("", ";P_{x}(e-) [GeV]", 600, -150., 150.), "e_MC_lvec_px", "weight")
h_e_py = df.Histo1D(("", ";P_{y}(e-) [GeV]", 600, -150., 150.), "e_MC_lvec_py", "weight")
h_e_pz = df.Histo1D(("", ";P_{z}(e-) [GeV]", 600, -150., 150.), "e_MC_lvec_pz", "weight")

h_p_px = df.Histo1D(("", ";P_{x}(e+) [GeV]", 600, -150., 150.), "p_MC_lvec_px", "weight")
h_p_py = df.Histo1D(("", ";P_{y}(e+) [GeV]", 600, -150., 150.), "p_MC_lvec_py", "weight")
h_p_pz = df.Histo1D(("", ";P_{z}(e+) [GeV]", 600, -150., 150.), "p_MC_lvec_pz", "weight")

h_CosTheta_W_minus = df.Histo1D(("", ";cos #theta_{W^{-}}", 40, -1., 1.), "CosTheta_W_minus", "weight")
h_CosTheta_W_plus = df.Histo1D(("", ";cos #theta_{W^{+}}", 40, -1., 1.), "CosTheta_W_plus", "weight")
# h_l_theta_prime = df.Histo1D("l_theta_prime", "weight")
h_l_plus_theta_prime = df.Histo1D("p_theta_prime", "weight")
h_l_plus_cosTheta_prime = df.Histo1D(("", ";cos #theta*_{e^{+}}", 40, -1., 1.), "p_cosTheta_prime", "weight")
h_l_plus_phi_prime = df.Histo1D(("", ";#phi*_{e+}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "p_phi_prime", "weight")
h_l_minus_theta_prime = df.Histo1D("e_theta_prime", "weight")
h_l_minus_cosTheta_prime = df.Histo1D(("", ";cos #theta*_{e^{-}}", 40, -1., 1.), "e_cosTheta_prime", "weight")
h_l_minus_phi_prime = df.Histo1D(("", ";#phi*_{e-}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "e_phi_prime", "weight")

h_l_cosTheta_prime = h_l_plus_cosTheta_prime.Clone()
h_l_cosTheta_prime.Add(h_l_minus_cosTheta_prime.GetPtr())
h_l_cosTheta_prime.SetTitle(";cos #theta*_{e^{#pm}}")
h_l_phi_prime = h_l_plus_phi_prime.Clone()
h_l_phi_prime.Add(h_l_minus_phi_prime.GetPtr())
h_l_phi_prime.SetTitle(";#phi*_{e#pm}")

In [24]:
# v_CosTheta_W_minus = ROOT.RDF.Experimental.VariationsFor(h_CosTheta_W_minus)
# v_l_cosTheta_prime = ROOT.RDF.Experimental.VariationsFor(h_l_cosTheta_prime)
# v_l_phi_prime = ROOT.RDF.Experimental.VariationsFor(h_l_phi_prime)
v_l_plus_cosTheta_prime = ROOT.RDF.Experimental.VariationsFor(h_l_plus_cosTheta_prime)
v_l_plus_phi_prime = ROOT.RDF.Experimental.VariationsFor(h_l_plus_phi_prime)
v_l_minus_cosTheta_prime = ROOT.RDF.Experimental.VariationsFor(h_l_minus_cosTheta_prime)
v_l_minus_phi_prime = ROOT.RDF.Experimental.VariationsFor(h_l_minus_phi_prime)

In [25]:
f_vars = ["fg1", "fz1", "fg2", "fz2", "fg3", "fz3", "fg5", "fz5"]
# vars = ["fg1", "fg2", "fg3"]
tgc_vars = ["dg1z", "dkg", "lz"]
weights = ["weight_alt1", "weight_alt2", "weight_alt3"]

columns = df.AsNumpy(f_vars + tgc_vars + weights)

In [26]:
f_vars_a = np.asarray([columns[v] for v in f_vars])
tgc_vars_a = np.asarray([columns[v] for v in tgc_vars])

In [27]:
f_vars_cov = np.cov(f_vars_a, bias=True)
tgc_vars_cov = np.cov(tgc_vars_a, bias=True)

In [28]:
print(f_vars_cov.shape)
print(np.round(1.e3 * f_vars_cov, decimals=2))

print(tgc_vars_cov.shape)
print(np.round(1.e3 * tgc_vars_cov, decimals=2))

(8, 8)
[[ 13.04  16.7  -12.66 -14.23 -21.02 -21.19   0.66   0.44]
 [ 16.7   26.64 -14.75 -22.98 -23.38 -36.4    0.16   0.37]
 [-12.66 -14.75  17.13  19.16  28.66  28.42  -1.     0.23]
 [-14.23 -22.98  19.16  31.83  29.91  51.2   -0.42   0.37]
 [-21.02 -23.38  28.66  29.91  66.09  66.78  -1.97   3.94]
 [-21.19 -36.4   28.42  51.2   66.78 121.9   -1.41   1.32]
 [  0.66   0.16  -1.    -0.42  -1.97  -1.41   5.39   8.04]
 [  0.44   0.37   0.23   0.37   3.94   1.32   8.04  16.67]]
(3, 3)
[[1286.55 1133.47  441.77]
 [1133.47 1022.78  380.37]
 [ 441.77  380.37  324.51]]


In [37]:
tgc = {}
inv_tgc_cov = np.linalg.inv(tgc_vars_cov)
print(inv_tgc_cov)

means_0 = {}
means_1 = {}

for o_i in tgc_vars:
    means_0[o_i] = np.average(columns[o_i])
    means_1[o_i] = np.average(columns[o_i], weights=columns["weight_alt1"])

for i, g_i in enumerate(tgc_vars):
    tgc[g_i] = 0.0
    for j, g_j in enumerate(tgc_vars):
        tgc[g_i] += inv_tgc_cov[i, j] * (means_1[g_j] - means_0[g_j])
    print(f"{g_i}: {tgc[g_i]} +/- {1 / np.sqrt(n_events * tgc_vars_cov[i, i])}")

# print(f"dg1z: {dg1z} +/- {dg1z_err}")

[[ 35.50817981 -37.89229492  -3.92297114]
 [-37.89229492  42.16981852   2.15465648]
 [ -3.92297114   2.15465648   5.89642561]]
dg1z: -0.04172149835671402 +/- 0.0008816321710422815
dkg: 0.0700935980072159 +/- 0.0009888029013060552
lz: 0.023175048059702025 +/- 0.001755435558584546


In [30]:
h_l_phi_prime.SetMinimum(0)
h_l_plus_phi_prime.SetMinimum(0)
h_l_minus_phi_prime.SetMinimum(0)

In [31]:
c_enu_m = ROOT.TCanvas()
h_enu_m.Draw()
c_enu_m.Draw()

c_pnu_m = ROOT.TCanvas()
h_pnu_m.Draw()
c_pnu_m.Draw()

In [32]:
c_CosTheta_W_minus = ROOT.TCanvas()
h_CosTheta_W_minus.Draw()
# v_CosTheta_W_minus["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
# v_CosTheta_W_minus["aTGC:g1z_up"].Draw("same")
# v_CosTheta_W_minus["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
# v_CosTheta_W_minus["aTGC:ka_up"].Draw("same")
# v_CosTheta_W_minus["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
# v_CosTheta_W_minus["aTGC:la_up"].Draw("same")
c_CosTheta_W_minus.Draw()

c_l_cosTheta_prime = ROOT.TCanvas()
h_l_cosTheta_prime.Draw()
# v_l_cosTheta_prime["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
# v_l_cosTheta_prime["aTGC:g1z_up"].Draw("same")
# v_l_cosTheta_prime["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
# v_l_cosTheta_prime["aTGC:ka_up"].Draw("same")
# v_l_cosTheta_prime["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
# v_l_cosTheta_prime["aTGC:la_up"].Draw("same")
c_l_cosTheta_prime.Draw()

c_l_phi_prime = ROOT.TCanvas()
h_l_phi_prime.Draw()
# v_l_phi_prime["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
# v_l_phi_prime["aTGC:g1z_up"].Draw("same")
# v_l_phi_prime["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
# v_l_phi_prime["aTGC:ka_up"].Draw("same")
# v_l_phi_prime["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
# v_l_phi_prime["aTGC:la_up"].Draw("same")
c_l_phi_prime.Draw()

c_l_plus_cosTheta_prime = ROOT.TCanvas()
h_l_plus_cosTheta_prime.Draw()
v_l_plus_cosTheta_prime["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_l_plus_cosTheta_prime["aTGC:g1z_up"].Draw("same")
v_l_plus_cosTheta_prime["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_l_plus_cosTheta_prime["aTGC:ka_up"].Draw("same")
v_l_plus_cosTheta_prime["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_l_plus_cosTheta_prime["aTGC:la_up"].Draw("same")
c_l_plus_cosTheta_prime.Draw()

c_l_plus_phi_prime = ROOT.TCanvas()
h_l_plus_phi_prime.Draw()
v_l_plus_phi_prime["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_l_plus_phi_prime["aTGC:g1z_up"].Draw("same")
v_l_plus_phi_prime["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_l_plus_phi_prime["aTGC:ka_up"].Draw("same")
v_l_plus_phi_prime["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_l_plus_phi_prime["aTGC:la_up"].Draw("same")
c_l_plus_phi_prime.Draw()

c_l_minus_cosTheta_prime = ROOT.TCanvas()
h_l_minus_cosTheta_prime.Draw()
v_l_minus_cosTheta_prime["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_l_minus_cosTheta_prime["aTGC:g1z_up"].Draw("same")
v_l_minus_cosTheta_prime["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_l_minus_cosTheta_prime["aTGC:ka_up"].Draw("same")
v_l_minus_cosTheta_prime["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_l_minus_cosTheta_prime["aTGC:la_up"].Draw("same")
c_l_minus_cosTheta_prime.Draw()

c_l_minus_phi_prime = ROOT.TCanvas()
h_l_minus_phi_prime.Draw()
v_l_minus_phi_prime["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_l_minus_phi_prime["aTGC:g1z_up"].Draw("same")
v_l_minus_phi_prime["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_l_minus_phi_prime["aTGC:ka_up"].Draw("same")
v_l_minus_phi_prime["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_l_minus_phi_prime["aTGC:la_up"].Draw("same")
c_l_minus_phi_prime.Draw()

In [33]:
c_aw = ROOT.TCanvas()
h_aw.Draw()
v_aw["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_aw["aTGC:g1z_up"].Draw("same")
v_aw["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_aw["aTGC:ka_up"].Draw("same")
v_aw["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_aw["aTGC:la_up"].Draw("same")
c_aw.Draw()
c_aw.SaveAs(f"{oo_plot_name}(")

c_awph = ROOT.TCanvas()
h_awph.Draw()
v_awph["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_awph["aTGC:g1z_up"].Draw("same")
v_awph["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_awph["aTGC:ka_up"].Draw("same")
v_awph["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_awph["aTGC:la_up"].Draw("same")
c_awph.Draw()
c_awph.SaveAs(oo_plot_name)

c_abph = ROOT.TCanvas()
h_abph.Draw()
v_abph["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_abph["aTGC:g1z_up"].Draw("same")
v_abph["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_abph["aTGC:ka_up"].Draw("same")
v_abph["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_abph["aTGC:la_up"].Draw("same")
c_abph.Draw()
c_abph.SaveAs(oo_plot_name)

c_dg1z = ROOT.TCanvas()
h_dg1z.Draw()
v_dg1z["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_dg1z["aTGC:g1z_up"].Draw("same")
v_dg1z["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_dg1z["aTGC:ka_up"].Draw("same")
v_dg1z["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_dg1z["aTGC:la_up"].Draw("same")
c_dg1z.Draw()
c_dg1z.SaveAs(oo_plot_name)

c_dkg = ROOT.TCanvas()
h_dkg.Draw()
v_dkg["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_dkg["aTGC:g1z_up"].Draw("same")
v_dkg["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_dkg["aTGC:ka_up"].Draw("same")
v_dkg["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_dkg["aTGC:la_up"].Draw("same")
c_dkg.Draw()
c_dkg.SaveAs(oo_plot_name)

c_lz = ROOT.TCanvas()
h_lz.Draw()
v_lz["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_lz["aTGC:g1z_up"].Draw("same")
v_lz["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_lz["aTGC:ka_up"].Draw("same")
v_lz["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_lz["aTGC:la_up"].Draw("same")
c_lz.Draw()
c_lz.SaveAs(oo_plot_name)


Info in <TCanvas::Print>: pdf file OO_wz.pdf has been created using the current canvas
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf


In [34]:

c_fg1 = ROOT.TCanvas()
h_fg1.Draw()
v_fg1["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fg1["aTGC:g1z_up"].Draw("same")
v_fg1["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fg1["aTGC:ka_up"].Draw("same")
v_fg1["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fg1["aTGC:la_up"].Draw("same")
c_fg1.Draw()
c_fg1.SaveAs(oo_plot_name)

c_fg2 = ROOT.TCanvas()
h_fg2.Draw()
v_fg2["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fg2["aTGC:g1z_up"].Draw("same")
v_fg2["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fg2["aTGC:ka_up"].Draw("same")
v_fg2["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fg2["aTGC:la_up"].Draw("same")
c_fg2.Draw()
c_fg2.SaveAs(oo_plot_name)

c_fg3 = ROOT.TCanvas()
h_fg3.Draw()
v_fg3["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fg3["aTGC:g1z_up"].Draw("same")
v_fg3["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fg3["aTGC:ka_up"].Draw("same")
v_fg3["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fg3["aTGC:la_up"].Draw("same")
c_fg3.Draw()
c_fg3.SaveAs(oo_plot_name)

c_fg4 = ROOT.TCanvas()
h_fg4.Draw()
v_fg4["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fg4["aTGC:g1z_up"].Draw("same")
v_fg4["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fg4["aTGC:ka_up"].Draw("same")
v_fg4["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fg4["aTGC:la_up"].Draw("same")
c_fg4.Draw()
c_fg4.SaveAs(oo_plot_name)

c_fg5 = ROOT.TCanvas()
h_fg5.Draw()
v_fg5["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fg5["aTGC:g1z_up"].Draw("same")
v_fg5["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fg5["aTGC:ka_up"].Draw("same")
v_fg5["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fg5["aTGC:la_up"].Draw("same")
c_fg5.Draw()
c_fg5.SaveAs(oo_plot_name)

c_fg6 = ROOT.TCanvas()
h_fg6.Draw()
v_fg6["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fg6["aTGC:g1z_up"].Draw("same")
v_fg6["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fg6["aTGC:ka_up"].Draw("same")
v_fg6["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fg6["aTGC:la_up"].Draw("same")
c_fg6.Draw()
c_fg6.SaveAs(oo_plot_name)

c_fg7 = ROOT.TCanvas()
h_fg7.Draw()
v_fg7["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fg7["aTGC:g1z_up"].Draw("same")
v_fg7["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fg7["aTGC:ka_up"].Draw("same")
v_fg7["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fg7["aTGC:la_up"].Draw("same")
c_fg7.Draw()
c_fg7.SaveAs(oo_plot_name)

c_fz1 = ROOT.TCanvas()
h_fz1.Draw()
v_fz1["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fz1["aTGC:g1z_up"].Draw("same")
v_fz1["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fz1["aTGC:ka_up"].Draw("same")
v_fz1["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fz1["aTGC:la_up"].Draw("same")
c_fz1.Draw()
c_fz1.SaveAs(oo_plot_name)

c_fz2 = ROOT.TCanvas()
h_fz2.Draw()
v_fz2["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fz2["aTGC:g1z_up"].Draw("same")
v_fz2["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fz2["aTGC:ka_up"].Draw("same")
v_fz2["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fz2["aTGC:la_up"].Draw("same")
c_fz2.Draw()
c_fz2.SaveAs(oo_plot_name)

c_fz3 = ROOT.TCanvas()
h_fz3.Draw()
v_fz3["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fz3["aTGC:g1z_up"].Draw("same")
v_fz3["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fz3["aTGC:ka_up"].Draw("same")
v_fz3["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fz3["aTGC:la_up"].Draw("same")
c_fz3.Draw()
c_fz3.SaveAs(oo_plot_name)

c_fz4 = ROOT.TCanvas()
h_fz4.Draw()
v_fz4["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fz4["aTGC:g1z_up"].Draw("same")
v_fz4["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fz4["aTGC:ka_up"].Draw("same")
v_fz4["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fz4["aTGC:la_up"].Draw("same")
c_fz4.Draw()
c_fz4.SaveAs(oo_plot_name)

c_fz5 = ROOT.TCanvas()
h_fz5.Draw()
v_fz5["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fz5["aTGC:g1z_up"].Draw("same")
v_fz5["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fz5["aTGC:ka_up"].Draw("same")
v_fz5["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fz5["aTGC:la_up"].Draw("same")
c_fz5.Draw()
c_fz5.SaveAs(oo_plot_name)

c_fz6 = ROOT.TCanvas()
h_fz6.Draw()
v_fz6["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fz6["aTGC:g1z_up"].Draw("same")
v_fz6["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fz6["aTGC:ka_up"].Draw("same")
v_fz6["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fz6["aTGC:la_up"].Draw("same")
c_fz6.Draw()
c_fz6.SaveAs(oo_plot_name)

c_fz7 = ROOT.TCanvas()
h_fz7.Draw()
v_fz7["aTGC:g1z_up"].SetMarkerColor(ROOT.kRed)
v_fz7["aTGC:g1z_up"].Draw("same")
v_fz7["aTGC:ka_up"].SetMarkerColor(ROOT.kBlue)
v_fz7["aTGC:ka_up"].Draw("same")
v_fz7["aTGC:la_up"].SetMarkerColor(ROOT.kGreen)
v_fz7["aTGC:la_up"].Draw("same")
c_fz7.Draw()
c_fz7.SaveAs(f"{oo_plot_name})")

Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file OO_wz.pdf and file closed
